# Purpose of this notebook:
The purpose of this notebook is to create a brainplot for the recall interval

Made March 6 2018

In [1]:
#Plotting!
%matplotlib notebook
from matplotlib import pyplot as plt
import matplotlib as mpl

# General Imports
import os, sys
from glob import glob # Glob.glob is more fun to say
from time import time 
import cluster_helper.cluster  # Parallelize by the same old antics

# Add some dimension to your life, np=arr, pd=labeled arr, xr=labeled N-D arr
import numpy as np # I say numpy like 'lumpy', no I don't mean num-pie
import pandas as pd # For Undergrads that say "But matlab makes it so much easier to view things"
import xarray as xr # N-D labels!

# Pizza ptsa stuff
from ptsa.data.TimeSeriesX import TimeSeriesX
from ptsa.data.readers import BaseEventReader, TalReader, EEGReader, JsonIndexReader

# EEG Toolbox NEW WAY OF IMPORTING!!!!
sys.path.append('/home2/loganf/SecondYear/Functions/CML/')
from GetData import get_sub_events, get_intrusions, get_sub_tal, get_sub_eeg, get_subs
from Utility.FrequencyCreator import logspace 
from SpectralAnalysis.RollingAverage import sliding_mean_fast
#from Utility.InclusionHelperFunctions import set_new_fields, apply_inclusion_criteria, get_valid_trials 
#from Utility.InclusionHelperFunctions import get_roi_indexes, morlet, 
from Utility.InclusionHelperFunctions import append_field_workaround
from Utility.RetrievalDeliberationCreation import get_matched_retrievals
from Stats.ZScoreFromWholeSession import BurkeZscoreNormalization
#from SpectralAnalysis.MentalChronometry import MentalChronometry
#from Utility.ZScoreMethods import ZScoreMethods


#Plotting!
%matplotlib notebook
from matplotlib import pyplot as plt
import matplotlib as mpl

# General Imports
import os, sys

from glob import glob # Glob.glob is more fun to say
from time import time 
import cluster_helper.cluster  # Parallelize by the same old antics

# EEG Toolbox NEW WAY OF IMPORTING!!!!
sys.path.append('/home2/loganf/SecondYear/Functions/CML/')
from GetData import get_sub_events, get_intrusions, get_sub_tal, get_sub_eeg, get_subs
from BrainPlots.BrainActivityWriter import Label
from BrainPlots.SphereBrainParc import get_avg_surface_spheres, get_avg_surface_spheres_ranges

Could not import MorletWaveletFilterCppLegacy (single-core C++ version of MorletWaveletFilter): cannot import name MorletWaveletTransform
You can still use MorletWaveletFilter


In [3]:
def Load_Data(savedfilename):
    """
    The purpose of this code is to be able to quickly load any kind of data
    using pickle.
    """
    import pickle

    try:

        with open(savedfilename, 'rb') as handle:
            loaded_data = pickle.load(handle)
        
        return loaded_data
    
    except:
        import numpy as np
        loaded_data = np.load(savedfilename)
        return loaded_data
    
def load_spheres_correct_index(subject, bp):
    
    #channel_paths = '/scratch/loganf/electrode_information/{}/{}.npy'.format(subject, 'bp')   
    #if os.path.exists(channel_paths): 
        #bp = np.load(channel_paths)
        
    bp_to_index = {i:bp[i] for i in xrange(len(bp))}
    sphere_path = '/scratch/loganf/spheres/{}_FR1'.format(subject)
    sphere_path = Load_Data(sphere_path)
    
    for k,v in enumerate(sphere_path):
        sphere_path[v] = np.array([bp_to_index[j] for i,j in enumerate(sphere_path[v]) if j in bp_to_index.keys()])
    return sphere_path

def update_spheres(sphere_d, data):
    new_d = {}
    for k,v in enumerate(sphere_d):
        in_sphere = data.sel(bipolar_pairs=sphere_d[v])
        if len(in_sphere.data) > 0:
            new_d[v] = in_sphere.data
        #else:
            #sphere_d.pop(v)
        #if in_sphere.data.shape == (200,):
            #sphere_d[v] = in_sphere.data
        #else:
            #sphere_d[v] = np.nanmean(in_sphere.data,0)
    return new_d

In [2]:
from os import path as op
import time
import numpy as np
from nibabel.freesurfer.io import read_geometry
from six import b

from scipy import sparse
from scipy.sparse import coo_matrix, csr_matrix

import nibabel as nib
import os


class Label(object):
    """Code Written by Jim Kregal for writing annotation labels to the surface of an averaged suubject brain"""

    def __init__(self, tal, pos=None, values=None, hemi=None, comment="",
                 name=None, outdir=None, subject=None, verbose=None,
                 basedir='/data/eeg/freesurfer/subjects/', radius=3.0,
                 stat=None, smooth=0.0, x=None, y=None, z=None):
        if hemi == 'lh':
            self.surf = basedir + 'average/surf/lh.pial'

        elif hemi == 'rh':
            self.surf = basedir + 'average/surf/rh.pial'

        if x is None:
            x = tal['atlases']['avg.dural']['x']

        if y is None:
            y = tal['atlases']['avg.dural']['y']

        if z is None:
            z = tal['atlases']['avg.dural']['z']

        pos = np.vstack((x, y, z)).T

        # need to filter all the electrodes by hemisphere
        if hemi == 'lh':
            stat = stat[pos[:, 0] < 0]
            pos = pos[pos[:, 0] < 0]
        elif hemi == 'rh':
            stat = stat[pos[:, 0] > 0]
            pos = pos[pos[:, 0] > 0]

        # TODO: may want to exclude depth electrodes
        # TODO: may want to have minimum distance to surface

        self.surface = nib.freesurfer.read_geometry(os.path.join(basedir, 'average' + '/surf/' + hemi + '.pial'))
        self.cortex = np.sort(
            nib.freesurfer.read_label(os.path.join(basedir, 'average' + '/label/' + hemi + '.cortex.label')))
        self.sulc = nib.freesurfer.read_morph_data(os.path.join(basedir, 'average' + '/surf/lh.sulc'))

        self.pos = pos
        self.stat = stat

        self.radius = radius
        self.hemi = hemi
        self.comment = comment
        self.verbose = verbose
        self.subject = subject
        self.name = name
        self.outdir = outdir
        self.smooth = smooth / 2.35482

    def map_stat_to_label(self):

        self.coords, faces, vertices = self.get_vertices_from_surf()

        fnames = []
        elec_vertices = np.empty(self.pos.shape[0], dtype=int)

        filename = op.join(self.outdir, self.subject)
        path_head, name = op.split(filename)
        if name.endswith('.label'):
            name = name[:-6]
        if not (name.startswith(self.hemi) or name.endswith(self.hemi)):
            name += '-' + self.hemi
        filename = op.join(path_head, name) + '.label'

        start_time = time.time()

        self.counts = np.full(len(vertices), 0)
        self.vertices = vertices
        self.values = np.full(len(vertices), 0.)

        connectivity = mesh_dist(self.surface[1], self.surface[0])

        for i, elec_coord in enumerate(self.pos):
            elec_vertices[i] = self.get_nearest_vertex(elec_coord)[0]

        elec_dists = sparse.csgraph.dijkstra(connectivity, indices=elec_vertices)

        for i, e_dist in enumerate(elec_dists):
            self.values[np.where(((e_dist < self.radius) & (e_dist > 0.0)) | (vertices == elec_vertices[i]))] += \
            self.stat[i]
            self.counts[np.where(((e_dist < self.radius) & (e_dist > 0.0)) | (vertices == elec_vertices[i]))] += 1

        self.values = self.values / self.counts
        # TODO: may want to remove vertices, pos, values where there is no data to make a smaller label files
        label = write_label(filename, self)
        fnames.append(filename)

        return fnames

    def get_vertices_from_surf(self):
        '''

        :param hemi:
        :return:
        '''

        coords, faces = read_geometry(self.surf)
        vertices = np.arange(coords.shape[0])
        return coords, faces, vertices

    def get_nearest_vertex(self, elec_coord):
        '''

        :return:
        '''

        dist = np.linalg.norm(self.coords - elec_coord, axis=1)
        idx = np.where(dist == dist.min())[0]

        return idx


def write_label(filename, label, verbose=None):
    """Write a FreeSurfer label.
    Parameters
    ----------
    filename : string
        Path to label file to produce.
    label : Label
        The label object to save.
    verbose : bool, str, int, or None
        If not None, override default verbose level (see :func:`mne.verbose`
        and :ref:`Logging documentation <tut_logging>` for more).
    Notes
    -----
    Note that due to file specification limitations, the Label's subject and
    color attributes are not saved to disk.
    See Also
    --------
    write_labels_to_annot
    """

    with open(filename, 'wb') as fid:
        n_vertices = len(label.vertices)
        data = np.zeros((n_vertices, 5), dtype=np.float)
        data[:, 0] = label.vertices
        data[:, 1:4] = label.coords  # self.pos #1e3 *
        data[:, 4] = label.values
        fid.write(b("#%s\n" % label.comment))
        fid.write(b("%d\n" % n_vertices))
        for d in data:
            fid.write(b("%d %f %f %f %f\n" % tuple(d)))
    return label


def average_duplicates(vertices, pos, values):
    result_vertices = np.unique(vertices)
    result_values = np.empty(result_vertices.shape)
    result_pos = np.empty((1, len(result_vertices), 3))

    for i, vertex in enumerate(result_vertices):
        result_values[i] = np.mean(values[vertices == vertex])
        result_pos[0, i, :] = np.mean(pos[0, vertices == vertex, :], axis=0)

    return result_vertices, result_pos, result_values


def gaussian(dist, sig):
    return np.exp(-np.power(dist, 2.) / (2 * np.power(sig, 2.)))


def mesh_edges(tris):
    """Return sparse matrix with edges as an adjacency matrix.
    Parameters
    ----------
    tris : array of shape [n_triangles x 3]
        The triangles.
    Returns
    -------
    edges : sparse matrix
        The adjacency matrix.
    """
    if np.max(tris) > len(np.unique(tris)):
        raise ValueError('Cannot compute connectivity on a selection of '
                         'triangles.')

    npoints = np.max(tris) + 1
    ones_ntris = np.ones(3 * len(tris))

    a, b, c = tris.T
    x = np.concatenate((a, b, c))
    y = np.concatenate((b, c, a))
    edges = coo_matrix((ones_ntris, (x, y)), shape=(npoints, npoints))
    edges = edges.tocsr()
    edges = edges + edges.T
    return edges


def mesh_dist(tris, vert):
    """Compute adjacency matrix weighted by distances.
    It generates an adjacency matrix where the entries are the distances
    between neighboring vertices.
    Parameters
    ----------
    tris : array (n_tris x 3)
        Mesh triangulation
    vert : array (n_vert x 3)
        Vertex locations
    Returns
    -------
    dist_matrix : scipy.sparse.csr_matrix
        Sparse matrix with distances between adjacent vertices
    """
    edges = mesh_edges(tris).tocoo()

    # Euclidean distances between neighboring vertices
    dist = np.linalg.norm(vert[edges.row, :] - vert[edges.col, :], axis=1)
    dist_matrix = csr_matrix((dist, (edges.row, edges.col)), shape=edges.shape)
    return dist_matrix


def Load_Data(savedfilename):
    """
    The purpose of this code is to be able to quickly load any kind of data
    using pickle.
    """
    import pickle

    try:

        with open(savedfilename, 'rb') as handle:
            loaded_data = pickle.load(handle)
            print
            'loaded successfully, fileloaded as as:\nloaded_data'
        return loaded_data
    except:
        import numpy as np
        loaded_data = np.load(savedfilename)
        return loaded_data

In [4]:
paths = glob('/scratch/loganf/spheres_single_time_point/*')

In [38]:
test_range = np.arange(-1, 1, .01)
start_stops = zip(test_range-.25, test_range+.5-.25)

In [46]:
start_stops[::50]

[(-1.25, -0.75),
 (-0.74999999999999956, -0.24999999999999956),
 (-0.24999999999999911, 0.25000000000000089),
 (0.25000000000000133, 0.75000000000000133)]

In [105]:
def filter_spheres_by_epoch(subject_spheres, epoch=0):
    new_d = {}
    for k,v in enumerate(subject_spheres):
        new_d[v] = np.mean(subject_spheres[v][:,epoch])
    return new_d

def update_dictionary(growing_dict, new_dict):
    for k,v in enumerate(new_dict):
        if v not in growing_dict:
            growing_dict[v] = []
        if v in growing_dict:
            growing_dict[v].append(new_dict[v])
    return growing_dict

from collections import OrderedDict
my_d = OrderedDict()

for p in paths:
    try:
        subject_spheres = Load_Data(p)
        subject_spheres = filter_spheres_by_epoch(subject_spheres, 0)
        update_dictionary(growing_dict = my_d, new_dict=subject_spheres)
    except:
        print(p)

In [106]:
test = OrderedDict()
for k,v in enumerate(my_d):
    if len(my_d[v]) > 5:
        test[v] = np.array(my_d[v])

In [108]:
test[test.keys()[0]]

array([-0.74794357, -0.1286965 , -0.81919502, -1.95961747,  0.6521079 ,
        0.18372324, -0.16267003, -2.29375731,  3.09857347, -2.05628947,
       -0.11216792,  0.157666  , -2.77440184,  0.05171172, -3.79001723,
        0.12442267])

In [109]:
from scipy.stats import ttest_1samp

In [117]:
p_values = OrderedDict()


for k,v in enumerate(test):
    t, p = ttest_1samp(test[v], 0)
    p_values[v] = p

In [120]:
p_test = np.array(p_values.values())

In [122]:
len(np.where(p_test < .05)[0])

2329

In [123]:
len(p_test)

43641

In [124]:
sys.path.append('/home2/loganf/statsmodels/')
from statsmodels.sandbox.stats.multicomp import fdrcorrection0 as fdr

In [125]:
boolean, corr = fdr(p_test)

In [127]:
np.where(corr<.05)

(array([], dtype=int64),)